<a href="https://colab.research.google.com/github/NataKiseleva/KL/blob/main/BPE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from collections import Counter

In [2]:
!pip install tokenizers
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics.pairwise import cosine_distances, cosine_similarity
from tokenizers import CharBPETokenizer, Tokenizer
from scipy.sparse import lil_matrix
from sklearn.model_selection import cross_val_score

     |████████████████████████████████| 3.3MB 8.3MB/s 


In [60]:
def byte_pairs(text, iterators= 10, pairs=100):
  d = set()
  symbols = '-'.join(list(text))
  for k in range(iterators):
    list_pair = []
    for i in range(len(text)):
        list_pair.append(''.join(text[i:i+2]))

    stat = Counter(list_pair)
    result = dict(stat.most_common(pairs)).keys()
    d.update(result)
    for x in result:
      e = len(x)//2
      a = x[:e]
      b = x[e:]
      c = a + '-' + b
      symbols = symbols.replace(c, a+'_'+b)

    text = ''.join(symbols.split('-'))
  symbols = symbols.replace('_', '')
  return symbols,d

In [15]:
text = open('lenta.txt', encoding= 'utf-8').read()[:4000]

In [61]:
tokens, d = byte_pairs(text)

In [ ]:
d

In [64]:
def tokenize(vocab, text):
  dict1 = {'-'.join(list(v)): v for v in vocab}
  chars = '-'.join(list(text))
  for e in dict1:
    chars = chars.replace(e, dict1[e])
  return chars

In [17]:
tokens

'Б-ои у С-опо-ц-кина и -Д-руск-еник закон-чились отступлением герман-цев-. Неприятель-, приблиз-ившись с севера к -О-сов-ц-у на-чал артиллерийск-у-ю борь-бу с крепость-ю-. В артиллерийском бо-ю принимают участие тя-желые калибры-. С- раннего у-тра 14 сентября огонь дости-г- зна-чительного напря-жения. Поп-ы-т-ка герман-ской пе-хоты пробиться бли-же к крепости отражена. В -Г-алиц-ии мы зан-яли -Д-ем-биц-у-. -Б-оль-шая колонна, отступавшая по -ш-оссе от П-еремы-ш-ля к С-анок-у-, об-стреливалась с вы-сот нашей батаре-ей и бежала, бросив пар-ки, обоз и ав-томобили. В-ылаз-ки г-арниз-она П-еремы-ш-ля остаются без-успешны-ми. При продол-жаю-щем-ся отступлении австрий-цев об-нару-жива-ется пол-но-е перемешивание их частей, за-х-ваты-ваются новые партии пленных-, ору-д-ия и про-чая матери-альная часть-. Н-а перевале -У-ж-ок мы раз-били неприятельский отря-д-, в-з-яли -его артиллери-ю и м-ного пленных и, продол-жая преследовать-, вступ-или в пределы В-ен-грии. -\n-«-Р-усский ин-валид-»-, 1-6- с

In [20]:
dictionary = set(tokens.split('-'))

In [21]:
len(dictionary)

319

In [23]:
max(dictionary, key = len)

'ся отступлении австрийцев обнаруживается полное перемешивание их частей, за'

In [24]:
tokens2 = byte_pairs(text, 5, 10)

In [25]:
dictionary2 = set(tokens2.split('-'))

In [26]:
len(dictionary2)

113

In [27]:
max(dictionary2, key = len)

'ли про'

In [28]:
tokens3 = byte_pairs(text, 5, 100)

In [29]:
dictionary3 = set(tokens3.split('-'))

In [30]:
len(dictionary3)

472

In [31]:
max(dictionary3, key = len)

', атаковал неприятеля и протаранил неприятельский ап'

In [32]:
tokens4 = byte_pairs(text, 50, 10)

In [33]:
dictionary4 = set(tokens4.split('-'))

In [34]:
len(dictionary4)

113

In [35]:
max(dictionary4, key = len)

'ли про'

In [36]:
text2 = open('lenta.txt', encoding= 'utf-8').read()[4000:8000]

In [65]:
result = tokenize(d, text2)

In [66]:
result

'ывший комен-дант К-алиша. -Э-то — здоровый детина, с ти-пи-ч-но прусским наг-лым лиц-ом и м-у-тными г-лазами. К-огда -этого «-зверя-» вели с партией пленных по улице и из тол-п-ы раздались крики и прок-лятия-, Прейскер- трусливо замешался в тол-пу пленных, стараясь скры-ться. -З-ато в зак-л-ю-чении, под- кон-воем наших сол-дат, Прейскер- принял пришед-шего к нем-у русского о-ф-ицера сид-я-, наг-ло развалившись в кресле. К-он-войные «-помог-ли-» -ем-у подняться и вы-тя-ну-ться во ф-ронт. К-роме Прейскера, нашими войсками вз-ят в плен и д-ру-гой «-ш-вабский зверь-» — капитан -Ш-мид-т, жив-ьем с-жи-гавший жителей деревень, запирая их в хата-х-. -Ж-урнал «-Н-ива-» №-3-8- от -2-0- сентября 1914 года-К-ак стало известно агентств-у -А-ссоши-э-й-тед- Пресс-, -э-ки-паж- последней -э-к-спедиц-ии на станц-ию -"-М-ир-"- с-читает -ее способной вый-ти из---под- контроля.-К-оман-дир- В-ик-тор- -А-ф-анасьев сказал-:- -"-М-ы ч-у-вств-у-ем себ-я хорош-о-, но грустим-, ч-то оставили станц-ию летать в бе

In [67]:
dictionary5 = set(result.split('-'))

In [68]:
len(dictionary5)

447

In [69]:
max(dictionary5, key = len)

'ка может привести как к потере топлива, так и к потере ориента'

Длина словаря стала меньше, чем для первоначального текста для таких же параметров, но словарь все еще подходит 

In [37]:
data = pd.read_csv('dataset_ok.csv')

In [38]:
train_texts, test_texts, train_labels, test_labels = train_test_split(data.text, data.label, 
                                                                      test_size=0.1, shuffle=True, random_state=21)

In [39]:
data['text'].to_csv('text3.txt', index=None)

In [41]:
token = CharBPETokenizer()
token.train('text3.txt')
token.save('train')
token = Tokenizer.from_file("train")

In [52]:
def make_matrix(file):   
  docs = file.to_list()
  docs_tokenize = []
  for doc in docs:
    s = token.encode(doc).ids
    docs_tokenize.append(s)

  #idf
  all_words = set([item for elem in docs_tokenize for item in elem])
  dict_words = Counter()
  for x in all_words:
    for doc in docs_tokenize:
      if x in doc:
        dict_words[x] += 1
  dict_words = {k: np.log(len(docs_tokenize)/dict_words[k]) for k in dict_words}

  tf_idf = []
  for doc in docs_tokenize:
    d = {}
    for word in doc:
      cnt = doc.count(word)/len(doc)
      d[word] = cnt*dict_words[word]
    tf_idf.append(d)

  matrix_text = lil_matrix((len(docs), token.get_vocab_size()))
  
  for i, x in enumerate(tf_idf):
    for y in x.keys():
      matrix_text[i, y] = tf_idf[i][y]
  
  return matrix_text

In [54]:
X_train = make_matrix(train_texts)
X_test = make_matrix(test_texts)
y_train = np.array(train_labels)
y_test = np.array(test_labels)

In [55]:
classificstion = SGDClassifier(loss="log", max_iter=30, alpha=0.0001, class_weight='balanced', n_jobs=-1)

classificstion.fit(X_train, y_train)
preds = classificstion.predict(X_test)

In [57]:
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

      INSULT       0.90      0.51      0.65       869
      NORMAL       0.93      0.99      0.96      6091
   OBSCENITY       0.82      0.65      0.73        72
      THREAT       0.84      0.73      0.78       167

    accuracy                           0.92      7199
   macro avg       0.87      0.72      0.78      7199
weighted avg       0.92      0.92      0.91      7199



In [58]:
cross_val_score(classificstion, X_train, y_train, scoring="f1_micro")

array([0.92004939, 0.91364408, 0.91688532, 0.9161843 , 0.92235857])

In [59]:
cross_val_score(classificstion, X_train, y_train, scoring="f1_macro")

array([0.74922404, 0.73048798, 0.73222906, 0.73813561, 0.74290294])